In [1]:
# підключення потрібних модулей
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pathlib import Path
import shutil
from pprint import pprint

# параметри виведення
pd.set_option("display.max_columns", 500) # кількість колонок
pd.set_option("display.max_rows", 1000) # кількість рядків
pd.set_option("display.max_colwidth", 300) # ширина колонок
pd.set_option("display.precision", 5) # кількість знаків після коми

# вимикаємо зайві попередження
import warnings
warnings.filterwarnings("ignore")

# друк всіх результатів в одній комірці а не тільки останнього
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# магічний метод для того щоб отримувати графіки біля комірок з кодом
%matplotlib inline

In [2]:
# завантажуємо необхідні компоненти Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, round, sum, lit, round

In [4]:
# Створюємо сесію Spark
spark = SparkSession.builder.appName("MyGoitSparkSandbox").getOrCreate()

In [5]:
# Завантажуємо датасет
products = spark.read.csv('products.csv', header=True)
purchases = spark.read.csv('purchases.csv', header=True)
users = spark.read.csv('users.csv', header=True)

# Виводимо перші 10 записів
products.show(2)
purchases.show(2)
users.show(2)

# перевіряємо створені датафрейми
products.printSchema()
purchases.printSchema()
users.printSchema()

+----------+------------+--------+-----+
|product_id|product_name|category|price|
+----------+------------+--------+-----+
|         1|   Product_1|  Beauty|  8.3|
|         2|   Product_2|    Home|  8.3|
+----------+------------+--------+-----+
only showing top 2 rows

+-----------+-------+----------+----------+--------+
|purchase_id|user_id|product_id|      date|quantity|
+-----------+-------+----------+----------+--------+
|          1|     52|         9|2022-01-01|       1|
|          2|     93|        37|2022-01-02|       8|
+-----------+-------+----------+----------+--------+
only showing top 2 rows

+-------+------+---+-----------------+
|user_id|  name|age|            email|
+-------+------+---+-----------------+
|      1|User_1| 45|user1@example.com|
|      2|User_2| 48|user2@example.com|
+-------+------+---+-----------------+
only showing top 2 rows

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = tr

In [6]:
# Рахуємо кількість рядків до видалення пустих значень
print('Quantity of rows before dropna:')
products.count()
purchases.count()
users.count()

# Видаляємо рядки з пустими значеннями
products = products.dropna()
purchases = purchases.dropna()
users = users.dropna()

# Рахуємо кількість рядків після видалення пустих значень
print('Quantity of rows after dropna:')
products.count()
purchases.count()
users.count()

Quantity of rows before dropna:


50

200

100

Quantity of rows after dropna:


47

195

95

In [7]:
# перейменовуємо ключові колонки
denormalized_df = purchases.withColumnRenamed('product_id', 'product_id_1').withColumnRenamed('user_id', 'user_id_1')
# приєднуємо довідники продуктів та юзерів
denormalized_df = denormalized_df.join(products, denormalized_df.product_id_1 == products.product_id, 'inner')
denormalized_df = denormalized_df.join(users, denormalized_df.user_id_1 == users.user_id, 'inner')
# видаляємо зайві колонки
denormalized_df = denormalized_df.drop(denormalized_df.user_id_1, denormalized_df.product_id_1)
# перевіряємо що вийшло
denormalized_df.show(3)
denormalized_df.printSchema()

+-----------+----------+--------+----------+------------+--------+-----+-------+-------+---+------------------+
|purchase_id|      date|quantity|product_id|product_name|category|price|user_id|   name|age|             email|
+-----------+----------+--------+----------+------------+--------+-----+-------+-------+---+------------------+
|          1|2022-01-01|       1|         9|   Product_9|  Beauty|  6.0|     52|User_52| 39|user52@example.com|
|          2|2022-01-02|       8|        37|  Product_37|Clothing|  6.0|     93|User_93| 25|user93@example.com|
|          3|2022-01-03|       1|        33|  Product_33|    Home|  9.4|     15|User_15| 30|user15@example.com|
+-----------+----------+--------+----------+------------+--------+-----+-------+-------+---+------------------+
only showing top 3 rows

root
 |-- purchase_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_name: string

In [8]:
# завдання 3 - Визначте загальну суму покупок за кожною категорією продуктів
denormalized_df \
    .withColumn('purchase_sum', (col('quantity') * col('price'))) \
    .select('category', 'purchase_sum').groupBy('category').sum() \
    .withColumnRenamed('sum(purchase_sum)', 'category_sum') \
    .show()

+-----------+------------------+
|   category|      category_sum|
+-----------+------------------+
|       Home|1438.8999999999999|
|     Sports|1755.4999999999998|
|Electronics|1141.8999999999999|
|   Clothing|             696.1|
|     Beauty|441.69999999999993|
+-----------+------------------+



In [9]:
# завдання 4 - Визначте суму покупок за кожною категорією продуктів для вікової категорії від 18 до 25 включно
denormalized_df.where((col('age') > 18) & (col('age') <= 25)) \
    .withColumn('purchase_sum', col('quantity') * col('price')) \
    .select('category', 'purchase_sum').groupBy('category').sum() \
    .withColumnRenamed('sum(purchase_sum)', 'category_sum') \
    .show()

+-----------+------------------+
|   category|      category_sum|
+-----------+------------------+
|       Home|174.70000000000002|
|     Sports|310.49999999999994|
|Electronics|             176.0|
|   Clothing|             245.0|
|     Beauty|41.400000000000006|
+-----------+------------------+



In [10]:
# завдання 5 - Визначте частку покупок за кожною категорією товарів від сумарних витрат для вікової категорії від 18 до 25 років
denormalized_df.where((col('age') > 18) & (col('age') <= 25)) \
    .withColumn('purchase_sum', col('quantity') * col('price')) \
    .select('category', 'purchase_sum').groupBy('category').sum() \
    .withColumnRenamed('sum(purchase_sum)', 'category_sum') \
    .withColumn('total_sum', lit(denormalized_df.where((col('age') > 18) & (col('age') <= 25)) \
    .agg(sum(col('quantity') * col('price'))).collect()[0][0])) \
    .withColumn('purchase_share_percent', round(col('category_sum') / col('total_sum') * 100, 2)) \
    .show()

+-----------+------------------+-----------------+----------------------+
|   category|      category_sum|        total_sum|purchase_share_percent|
+-----------+------------------+-----------------+----------------------+
|       Home|174.70000000000002|947.5999999999999|                 18.44|
|     Sports|310.49999999999994|947.5999999999999|                 32.77|
|Electronics|             176.0|947.5999999999999|                 18.57|
|   Clothing|             245.0|947.5999999999999|                 25.85|
|     Beauty|41.400000000000006|947.5999999999999|                  4.37|
+-----------+------------------+-----------------+----------------------+



In [11]:
# завдання 6 - Виберіть 3 категорії продуктів з найвищим відсотком витрат споживачами віком від 18 до 25 років
denormalized_df.where((col('age') > 18) & (col('age') <= 25)) \
    .withColumn('purchase_sum', col('quantity') * col('price')) \
    .select('category', 'purchase_sum').groupBy('category').sum() \
    .withColumnRenamed('sum(purchase_sum)', 'category_sum') \
    .withColumn('total_sum', lit(denormalized_df.where((col('age') > 18) & (col('age') <= 25)) \
    .agg(sum(col('quantity') * col('price'))).collect()[0][0])) \
    .withColumn('purchase_share_percent', round(col('category_sum') / col('total_sum') * 100, 2)) \
    .orderBy(col('purchase_share_percent').desc()) \
    .limit(3) \
    .show()

+-----------+------------------+-----------------+----------------------+
|   category|      category_sum|        total_sum|purchase_share_percent|
+-----------+------------------+-----------------+----------------------+
|     Sports|310.49999999999994|947.5999999999999|                 32.77|
|   Clothing|             245.0|947.5999999999999|                 25.85|
|Electronics|             176.0|947.5999999999999|                 18.57|
+-----------+------------------+-----------------+----------------------+



In [13]:
# Закриваємо сесію Spark
spark.stop()